# Requirements

In [ ]:
%pip install pandas numpy matplotlib seaborn 
%pip install scikit-learn
%pip install imbalanced-learn

### Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import plotly.express as px
import plotly.graph_objects as go
from scipy.stats import kstest, kruskal, norm

### Data

- Definição das funções

In [ ]:
DATA_PATH = 'PhiUSIIL_Phishing_URL_Dataset.csv'

def load_data(data_path=DATA_PATH):
    data = pd.read_csv(data_path)
    
    print("\nInformações do Dataset:")
    display(data.info())
    
    print("\nPrimeiras linhas do Dataset:")
    display(data.head(10))
    
    print("\nEstatísticas do Dataset:")
    display(data.describe())
    
    print("\nValores nulos no Dataset:")
    display(data.isnull().sum())
    
    # Modificar a a coluna label para target, para ser de fácil perceção ??
    data.rename(columns={'label': 'target'}, inplace=True)
    
    # Remover colunas categóricas
    categorical_columns = data.select_dtypes(include=['object']).columns
    data.drop(categorical_columns, axis=1, inplace=True)
    
    #Vai apresentar o nome das colunas
    #display("Columns: " + ", ".join(list(map(str, data.columns))))
    
    return data

In [ ]:
def separar_features_target(data):
    X = data.copy()
    X.drop('target', axis=1,inplace=True)
    y = data['target'].copy()
    
    return X,y

In [ ]:
def normalizar_features(X): # 
    scaler = StandardScaler() # É o mesmo que usar Z_score, pois a média é 0 e o desvio padrão é 1
    X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)
    return X

- Execução Pré - Processamento

In [ ]:
data = load_data()
X, y  = separar_features_target(data)
X = normalizar_features(X)

- Análise dos dados

  - 1 - Legítimo | 0 - Phishing

In [ ]:
def plot_class_distribution(data):
    
    values = data['target'].value_counts()
    keys = ['Legítimo', 'Phishing']
    
    fig_pie = px.pie(names=keys, values=values, title='Distribuição das classes (%)')
    
    fig_bar = px.bar(x = keys, y=values, labels={'x': 'Classe', 'y': 'Contagem'},
                     title="Distribuição das Classes (Phishing vs. Legítimo)")
    
    fig_pie.show()
    fig_bar.show()
    
    print("\nPerc. of class 1 (Legítimo): " + str(data['target'].sum() / data.shape[0] * 100) + " %")

    #São muitos features, como simplificar aqui para verificar a matriz de correlação?
    
    # Com matplotlib
    corr = data.corr()
    
    plt.figure(figsize=(40, 40))
    sns.heatmap(corr, annot=True, vmin=-1.0, cmap='mako')
    plt.title("Matriz de Correlação")
    plt.show()
    
    ''' 
    # Com plotly
    fig_corr = px.imshow(corr, title="Matriz de Correlação",
                     color_continuous_scale= 'blues',
                     labels={'x': 'Features', 'y': 'Features', 'color': 'Correlação'})
    fig_corr.show()
    '''

    
    
plot_class_distribution(data)

### Kolmogorov-Smirnov Test

1️⃣ Primeiro, usamos KS-Test no dataset completo para decidir se as features são normalmente distribuídas.

2️⃣ Depois, usamos KS-Test separado por classe para verificar se as distribuições variam entre phishing e legítimo.

3️⃣ Se houver uma diferença grande entre classes, a feature pode ser relevante para classificação.

In [ ]:

def KS_Test(X, y, features, alpha=0.05):
    
    #Separar por target e verificar se segue uma distribuição normal
    normal_features_target = {}
    non_normal_features_target = {}
    for f in features:
        for cls in [0, 1]:  # 1 = Legítimo, 0 = Phishing
            class_data = X[y == cls][f]
            class_data = (class_data - class_data.mean()) / class_data.std()  # Normalização Z-score
            
            ks_stat, p_value = kstest(class_data, 'norm')
            #print(f"Feature: {f}, Classe: {cls}, KS Statistic: {ks_stat}, p-value: {p_value}")
            if p_value > alpha:
                normal_features_target[f] = cls
            else:
                non_normal_features_target[f] = cls
                     
            
    #Sem separar por target
    ks_statistic = []
    p_value = []
    
    normal_features = []
    non_normal_features = []
    
    for feature in features:
        ks, p= kstest(X[feature], 'norm')
        ks_statistic.append(ks)
        p_value.append(p)
        if p > alpha:
            normal_features.append(feature)
        else:
            non_normal_features.append(feature)
            
    print(features)
    print(ks_statistic)
    print(p_value)
    
    return normal_features, non_normal_features
    
    
# Não sei se faz muito sentido ter um plot     


# Teste de Kolmogorov-Smirnov detalhado com visualização
def ks_test_with_plot(X, y, feature):
    plt.figure(figsize=(12, 6))
    
    plt.subplot(1, 2, 1)
    sns.histplot(X[feature], kde=True, stat="density", linewidth=0, bins=30)
    xmin, xmax = plt.xlim()
    x = np.linspace(xmin, xmax, 100)
    p = norm.pdf(x, X[feature].mean(), X[feature].std())
    plt.plot(x, p, 'r', linewidth=2, label='Normal PDF')
    plt.title(f"Histograma para {feature} com PDF Teórica")
    plt.legend()
    
    plt.subplot(1, 2, 2)
    sns.ecdfplot(X[feature], label='Empirical CDF')
    plt.plot(x, norm.cdf(x, X[feature].mean(), X[feature].std()), 'r-', label='Theoretical CDF')
    plt.title(f"Empirical vs Theoretical CDF para {feature}")
    plt.legend()
    
    plt.show()
    
    # Executar KS-Test
    ks_stat, p_value = kstest(X[feature], 'norm', args=(X[feature].mean(), X[feature].std()))
    print(f"Feature: {feature}, KS Statistic: {ks_stat}, p-value: {p_value}")
        
        
            
            
normal_features, non_normal_features = KS_Test(X, y, X.columns)
# Nenhuma feature segue uma distribuição normal (p-value < 0.05)
print("Features Normal Distribution : ", normal_features)
print("Features Non Normal Distribution: ", non_normal_features)


### Kruskal Wallis Test

In [ ]:

def KW_test(data, features):
    Hs = {}
    
    ix_legitimo = np.where(y == 1)[0]
    ix_phishing = np.where(y == 0)[0]
   
    for i in range(X.shape[1]):
        stat = kruskal(X.iloc[ix_legitimo, i], X.iloc[ix_phishing, i]) # iloc trabalha com pandas, flatten trabalha com numpy
        Hs[features[i]] = stat  
 
    Hs_sorted = sorted(Hs.items(), key=lambda x: x[1], reverse=True)
    
    return Hs_sorted



Hs_sorted = KW_test(data, X.columns)
print(Hs_sorted)
    

In [ ]:
def plot_features(X, y, top_features):
    data = X.copy()
    data['Class'] = y.map({0: 'Phishing', 1:'Legítimo'})
    
    color_discrete_map = {'Legítimo': 'lightskyblue', 'Phishing': 'hotpink'}
    
    for feature in top_features:
        fig = px.violin(data, y=feature, x='Class', box=True, points="all", 
                        title=f'Violin plot for {feature}', color='Class', 
                        color_discrete_map=color_discrete_map)
        fig.show()
        
plot_features(X, y, [x[0] for x in Hs_sorted[:1]])